# Импорты

In [17]:
import os
import random
import sys
import json
import pandas as pd
import chromadb
from chromadb.config import Settings
from concurrent.futures import ThreadPoolExecutor

sys.path.append('..')
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(os.path.abspath(os.path.join('..', '.env')))

True

# DeepSeek

In [3]:
API_KEY = os.environ["DEEPSEEK_API_KEY"]
BASE_URL = "https://api.deepseek.com"

deep_seek_client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# Чтение датасета

In [4]:
df = pd.read_csv('./data/merged_news_dataset.csv')
df.head()

,news,date,tags,title
0,Российские отели «всё включено» обяжут предлаг...,2023-03-10 00:00:00,NaN,NaN
1,""" Самолет (SMLT) в 4кв2023 собирается побить р...",2023-10-26 00:00:00,"['аналитика', 'факты']",NaN
2,"""Высокие цены на нефть, похоже, надолго. Мир в...",2022-08-29 00:00:00,"['ROSN', 'LKOH']",NaN
3,ЦБ определил порядок продажи заблокированных и...,2023-12-11 00:00:00,NaN,NaN
4,Чем привлекателен выпуск Газпром Капитала,2024-01-12 00:00:00,['Газпром капитал ЗО28-1-Е RU000A105BY1'],NaN


In [5]:
df_new = df.copy()

# Contextual Retrieval Preprocessing

In [6]:
SYSTEM_CONTEXTUAL_RETRIEVAL = """Ты - AI-аннотатор финансовых новостей, специализирующийся на анализе и интерпретации причинно-следственных связей в финансовых событиях. 
Твоя задача — анализировать пользовательское сообщение и предоставлять дополнительный контекст для улучшения поиска текста в векторной базе данных. 

### Используй следующие подсказки для корректного выполнения задачи:
1. В маленьких кусках текста — дополни контекст, раскрыв тему новости более подробно.
2. В больших текстах — сделай контекстуальное сжатие для более правильного описания новости.
3. Строго используй заданный формат вывода.

### Формат вывода:
{
    "description_chunk": Контекстуальное описание чанка. Для маленьких текстов — добавь детали, для больших — сделай сжатое резюме. Описание должно быть четким, информативным и содержать ключевые детали.
    "topic": Тема новости. Выбери одну из следующих категорий: акции, облигации, золото, нефть, криптовалюта, другое. Если тема не подходит ни под одну категорию, укажи "другое".
}

### Пример 1 (маленький текст):
Вход: "Цены на нефть выросли на 5%."
Выход: {
    "description_chunk": "Цены на нефть выросли на 5% из-за сокращения поставок со стороны крупных производителей, таких как ОПЕК+, и увеличения спроса на энергоносители в связи с сезонным ростом экономической активности.",
    "topic": "нефть"
}

### Пример 2 (большой текст):
Вход: "Компания XYZ объявила о выпуске новых акций для привлечения капитала. Это решение было принято в связи с необходимостью финансирования масштабного расширения производства. Однако инвесторы отреагировали на это негативно, что привело к падению стоимости акций на 10%."
Выход: {
    "description_chunk": "Компания XYZ выпустила новые акции для финансирования расширения производства, что вызвало негативную реакцию инвесторов и привело к падению стоимости акций на 10%.",
    "topic": "акции"
}

При создании описания придерживайся нейтрального тона, избегай субъективных оценок и сосредоточься на фактах и причинно-следственных связях:"""

In [ ]:
def process_news(sample):
    messages = [
        {"role": "system", "content": SYSTEM_CONTEXTUAL_RETRIEVAL},
        {"role": "user", "content": sample["news"]}
    ]

    output = deep_seek_client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False,
        temperature=1.5
    ).choices[0].message.content

    try:
        output = json.loads(output)
    except json.JSONDecodeError:
        output = "error"

    output |= {"date": sample["date"], "title": sample["title"]}
    print(output)

    return output

In [ ]:
outputs = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_news, sample) for _, sample in df_new.iloc[5450:, :].iterrows()]
    for future in futures:
        outputs.append(future.result())

print(outputs)

# Запись датасета

In [76]:
with open("./data/contextual_dataset.json", "w", encoding="utf-8") as f:
    json.dump(outputs, f, ensure_ascii=False)